<a href="https://colab.research.google.com/github/EsserMishelle/DataScience/blob/main/Credit_Card_Fraud_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import seaborn
import pandas as pd
import numpy as np
# import codecademylib3
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# import codecademylib3

# Load the data
transactions = pd.read_csv('transactions_modified.csv')

In [31]:
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 5000

In [32]:
print(transactions.head(20))
print(transactions.info())

    Unnamed: 0  step      type      amount     nameOrig  oldbalanceOrg  \
0            0   206  CASH_OUT    62927.08   C473782114           0.00   
1            1   380   PAYMENT    32851.57  C1915112886           0.00   
2            2   570  CASH_OUT  1131750.38  C1396198422     1131750.38   
3            3   184  CASH_OUT    60519.74   C982551468       60519.74   
4            4   162   CASH_IN    46716.01  C1759889425     7668050.60   
5            5   350   PAYMENT    16013.85  C2093859850       72766.00   
6            6   306  CASH_OUT   277310.34  C1480259234           0.00   
7            7   332  CASH_OUT   184157.68   C399464792      188588.19   
8            8   330   CASH_IN   136804.02   C304503510       20781.00   
9            9   259   CASH_IN    60609.99  C2089244493     2907876.74   
10          10   401  CASH_OUT   151635.96  C1084030527           0.00   
11          11   520   CASH_IN   466356.34    C52543009      100790.00   
12          12   308   PAYMENT    2704

In [34]:
# Summary statistics on amount column
transactions.describe()


,Unnamed: 0,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isPayment,isMovement,accountDiff
count,1000.000000,1000.000000,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1000.000000,1000.000000,1000.000000,1.000000e+03
mean,499.500000,280.664000,5.373080e+05,1.049284e+06,6.376146e+05,1.028848e+06,1.302326e+06,0.282000,0.220000,0.605000,1.744828e+06
std,288.819436,167.174593,1.423692e+06,3.226500e+06,2.717351e+06,2.678541e+06,3.038042e+06,0.450198,0.414454,0.489095,3.792962e+06
min,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00
25%,249.750000,164.000000,2.933705e+04,1.287500e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,4.273675e+04
50%,499.500000,261.000000,1.265305e+05,4.371400e+04,0.000000e+00,3.881927e+04,1.956572e+05,0.000000,0.000000,1.000000,3.383816e+05
75%,749.250000,373.250000,3.010378e+05,4.080914e+05,3.795663e+04,7.747167e+05,1.185959e+06,1.000000,0.000000,1.000000,1.759913e+06
max,999.000000,741.000000,1.000000e+07,5.039905e+07,4.039905e+07,3.085651e+07,3.183962e+07,1.000000,1.000000,1.000000,5.039905e+07


In [35]:
# How many fraudulent transactions?
# fraud_count = transactions.isFraud.value_counts()
# fraud_count = transactions[transactions['isFraud'] > 0.0].value_counts()
fraud_count = transactions['isFraud'].value_counts()

print(fraud_count)

isFraud
0    718
1    282
Name: count, dtype: int64


In [36]:
transactions.corr(numeric_only= True)

,Unnamed: 0,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isPayment,isMovement,accountDiff
Unnamed: 0,1.000000,-0.014241,-0.007378,-0.046280,-0.053385,-0.003873,-0.012985,0.027568,0.039696,0.028249,-0.025400
step,-0.014241,1.000000,0.130207,0.055953,-0.010586,-0.046148,-0.041700,0.351962,-0.107143,0.169905,0.025575
amount,-0.007378,0.130207,1.000000,0.567740,0.148727,-0.045255,0.145692,0.434568,-0.195533,0.258010,0.437568
oldbalanceOrg,-0.046280,0.055953,0.567740,1.000000,0.895063,0.008752,0.062284,0.148822,-0.164140,-0.068597,0.736462
newbalanceOrig,-0.053385,-0.010586,0.148727,0.895063,1.000000,0.052707,0.014347,-0.079965,-0.115619,-0.225340,0.656636
oldbalanceDest,-0.003873,-0.046148,-0.045255,0.008752,0.052707,1.000000,0.883837,-0.127545,-0.197623,0.059987,0.609762
newbalanceDest,-0.012985,-0.041700,0.145692,0.062284,0.014347,0.883837,1.000000,-0.033300,-0.222012,0.141651,0.580150
isFraud,0.027568,0.351962,0.434568,0.148822,-0.079965,-0.127545,-0.033300,1.000000,-0.332833,0.506388,0.052265
isPayment,0.039696,-0.107143,-0.195533,-0.164140,-0.115619,-0.197623,-0.222012,-0.332833,1.000000,-0.657269,-0.232860
isMovement,0.028249,0.169905,0.258010,-0.068597,-0.225340,0.059987,0.141651,0.506388,-0.657269,1.000000,0.046162


In [37]:
transactions.corr(numeric_only= True)['isFraud'].sort_values(ascending = False)

,isFraud
isFraud,1.000000
isMovement,0.506388
amount,0.434568
step,0.351962
oldbalanceOrg,0.148822
accountDiff,0.052265
Unnamed: 0,0.027568
newbalanceDest,-0.033300
newbalanceOrig,-0.079965
oldbalanceDest,-0.127545


isFraud is most positively related to isMovement, amount, step and most negatively related to isPayment. However, using step can cause data leakage:

The model learns when fraud happens, not why fraud happens
In real life, we won’t know future steps. That’s why: step is informative but not causal.


For the amount column, if I look at the summary stats:I would discover that there

amount
min: 0
max: 50,000,000+


This causes:
Coefficients dominated by outliers
Linear models become unstable
Model performance looks great in training, poor in real life

In [38]:
pd.crosstab(transactions['type'], transactions['isMovement'])

isMovement,0,1
type,,
CASH_IN,175,0
CASH_OUT,0,400
DEBIT,6,0
PAYMENT,214,0
TRANSFER,0,205


In [39]:
pd.crosstab(transactions['type'], transactions['isPayment'])

isPayment,0,1
type,,
CASH_IN,175,0
CASH_OUT,400,0
DEBIT,0,6
PAYMENT,0,214
TRANSFER,205,0


In [40]:
pd.crosstab(transactions['type'], transactions['amount'])

amount,0.00,174.92,206.69,210.92,218.18,224.94,261.04,281.99,328.46,449.19,639.93,640.55,642.09,661.61,709.00,742.06,908.69,930.04,942.51,951.93,968.19,992.86,1004.50,1074.98,1087.10,1159.72,1281.01,1351.20,1468.19,1489.63,1527.22,1528.41,1531.42,1565.64,1721.46,1874.40,1913.52,1961.86,2132.50,2180.71,2272.61,2348.03,2369.01,2407.88,2428.45,2435.44,2539.53,2563.81,2651.81,2684.72,...,2374412.42,2406332.79,2426076.89,2451423.58,2514878.33,2656973.04,2698972.36,2728625.68,2737766.80,2763025.35,2828068.73,2923356.25,2983996.60,3120690.98,3156421.43,3283695.97,3297243.38,3309631.10,3354200.39,3571807.73,3640983.71,3642569.29,3832058.34,3857176.83,4130371.73,4177467.46,4262592.28,4505012.56,4559921.56,4892226.25,4895343.33,5021186.54,5131896.17,5166728.87,5460002.91,5566368.02,5926745.36,6188514.81,6501303.14,7044227.89,7218629.01,7428844.46,7515624.63,7782452.17,8317724.27,8931550.69,9198459.96,9639524.70,9960382.40,10000000.00
type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
CASH_IN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CASH_OUT,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,0,1,1,0,0,0,1,1,1,1,0,1,0,1,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,3
DEBIT,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
PAYMENT,0,0,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TRANSFER,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,1,0,0,1,1,1,1,1,0,1,1,0,0,1,1,1,0,1,1,1,0,1,1,0,1,0,1,1,0,1,1,5


In [42]:
# isPayment, isMovement and accountDiff are all int64 type so they don't need to be
# dummy-coded or type conversion. They are model ready.

# Create features and label variables
features = ['amount', 'isPayment', 'isMovement', 'accountDiff']
label= 'isFraud'

# Split dataset
X = transactions[features]
y = transactions[label]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [43]:
# Normalize/scaler scale the features variables
ss = StandardScaler()
X_train_ss = ss.fit_transform(X_train)
X_test_ss = ss.transform(X_test)

In [17]:
# Train the logistic regression on scaled training data
lr = LogisticRegression()
lr.fit(X_train_ss, y_train)


LogisticRegression()

In [18]:
# Score the model on the training data
lr.score(X_train_ss, y_train)

# Score the model on the test data
lr.score(X_test_ss, y_test)

print(f'Train accuracy: {lr.score(X_train_ss, y_train)}')
print(f'Test accuracy: {lr.score(X_test_ss, y_test)}')

# Print the model coefficients
print(f'The logistic model\'s coefficients: {lr.coef_}')


Train accuracy: 0.8385714285714285
Test accuracy: 0.85
The logistic model's coefficients: [[ 2.42110403 -0.61050379  2.10147921 -0.987915  ]]


In [24]:
# New transaction data
transaction1 = np.array([123456.78, 0.0, 1.0, 54670.1])
transaction2 = np.array([98765.43, 1.0, 0.0, 8524.75])
transaction3 = np.array([543678.31, 1.0, 0.0, 510025.5])
transaction4 = np.array([734104.8, 0.0, 1.0, 14975.6])

# Combine new transactions into a single array
my_transactions = np.stack((transaction1, transaction2, transaction3, transaction4))

# Turn this NumPy array into a DataFrame, to avoid the 'feature names are missing' warning from sklearn
my_transactions_df = pd.DataFrame(my_transactions,
                               columns= ['amount', 'isPayment', 'isMovement', 'accountDiff'])

# Normalize the new transactions
my_transactions_ss = ss.transform(my_transactions_df)

# Predict fraud on the new transactions
lr.predict(my_transactions_ss_df)
print(lr.predict(my_transactions_ss_df))
# Show probabilities on the new transactions
lr.predict_proba(my_transactions_ss_df)
# print('The probability of the fraud for these traction are: {lr.predict_proba(my_transactions_ss)[1]}')

NameError: name 'my_transactions_ss_df' is not defined